# Análise de Sentimentos

#### Victor Marcius Magalhães Pinto

In [1]:
%load_ext autoreload
%autoreload 2

from src.text_tokenize import TextTokenizer
import pandas as pd
import numpy as np
from os.path import join
from sklearn.model_selection import train_test_split
import click

In [2]:
df = pd.read_csv(join("..", 'data', 'processed', 'review_message.csv'))
df["message"] = df["message"].apply(lambda d: "none" if type(d) == float else d)
X = df.drop(['score'], axis=1)
y = df["score"]

X_train, X_test, y_train, y_test = train_test_split(X, y)


click.echo(f'Train size {X_train.shape[0]} Test size {X_test.shape[0]}')

Train size 75000 Test size 25000


In [35]:
tk = TextTokenizer(X_train, "message", use_gpu=True, n_tokens=200)

Using GPU for spacy.


In [36]:
import json
with open('..\word_map.json', 'w') as fp:
    json.dump(tk.map, fp, indent=4)